In [ ]:
!pip install pandas scikit-learn numpy

In [ ]:
import pandas as pd

train = pd.read_csv('train.csv')
oil = pd.read_csv('./oil.csv')

In [ ]:
def getOneStore(df, store_nbr):
    df = df[df['store_nbr'] == store_nbr]
    return df

train = getOneStore(train, 1)
train.head()

In [ ]:
def addRowWithSameDate(_to, _from, column):
    rows = _from['date']
    index = 0
    res = _to.copy()
    for row in rows:
        dateToFill = res[res['date'] == row].index
        for c in dateToFill:
            res.at[c, column] = _from.at[index, column]
        index += 1
    return res

updateTrain = addRowWithSameDate(train, oil, 'dcoilwtico')
updateTrain.head()

In [ ]:
holi_df = pd.read_csv('holidays_events.csv')

In [ ]:
def deleteTransferredTrue(df):
    if (df['transferred'].dtype == bool):
        x = 0
        for trans in df['transferred']:
            if trans:
                x += 1
        print('Number of transferred: ' + str(x))
        df = df.drop(df[df['transferred'] == True].index)
    return df

holi_df = deleteTransferredTrue(holi_df)

In [ ]:
def addHolidayWithSameDate(_to, _from):
    rows = _from['date']
    for row in rows:
        dateToFill = _to[_to['date'] == row].index
        from_index = _from[_from['date'] == row].index[0]
        for c in dateToFill:
            _to.at[c, 'holiday'] = _from.at[from_index, 'locale_name']
    return _to

updateTrain = addHolidayWithSameDate(updateTrain, holi_df)

In [ ]:
from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(updateTrain['holiday'])
updateTrain['holiday'] = le.transform(updateTrain['holiday'])

In [ ]:
updateTrain['dcoilwtico'] = updateTrain['dcoilwtico'].fillna(updateTrain['dcoilwtico'].median())

In [ ]:
updateTrain['family'] = le.fit_transform(updateTrain['family'])

In [ ]:
from sklearn.model_selection import train_test_split

X = updateTrain.drop(columns = ['sales', 'date'])
y = updateTrain['sales']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, test_size = 0.3)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
clf = DecisionTreeRegressor(max_depth = 10)
clf = clf.fit(X_train, y_train)

In [ ]:
predictions = clf.predict(X_test)
predictions

In [ ]:
from sklearn.metrics import r2_score
print(r2_score(predictions, y_test))